In [1]:
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
style.use('ggplot')
import seaborn as sns
import tqdm
import random

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
!pip install lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

data_dir = '/mnt/elice/dataset'

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [214]:
def set_seed(seed):
    # Set the random seed for PyTorch on CPU and GPU
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Set the random seed for NumPy
    np.random.seed(seed)

    # Set the random seed for Python's built-in random module
    random.seed(seed)

# Example usage:
random_seed = 42  # Choose any integer value as the random seed
set_seed(random_seed)

# Now, the random seeds are set, and you can proceed with your PyTorch code.

In [238]:
# train_df: serial number, timestamp, X1, X2~18, Y
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"), index_col='Serial Number')
# test_x: serial number, timestamp, X1, X2~18
test_x = pd.read_csv(os.path.join(data_dir, "test_x.csv"), index_col='Serial Number')

''' timestamp 열 형식 바꾸기 '''
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))
test_x['TIMESTAMP'] = pd.to_datetime(test_x['TIMESTAMP'])
test_x['TIMESTAMP'] = test_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

''' 컬럼 키 추출 '''
serial_key = train_df.index.name
date_time_key = list(train_df.columns)[0]
feature_keys = list(train_df.columns)[2:-1]
target_key = list(train_df.columns)[-1]

# train_x: serial number, timestamp, X1, X2~18
train_x = train_df.drop(columns='Y')
# train_y: serial_number, Y
train_y = pd.read_csv(os.path.join(data_dir, "train_y.csv"), index_col='Serial Number')

# 데이터 전처리

## 장비 이름을 나타내는 X1 변수를 제거

In [239]:
train_x.drop(columns='X1', inplace=True)
test_x.drop(columns='X1', inplace=True)

## `StandardScaler` 를 활용해 표준화 

In [240]:
scaler = StandardScaler()

train_x[feature_keys] = scaler.fit_transform(train_x[feature_keys])
test_x[feature_keys] = scaler.transform(test_x[feature_keys])

## Serial Number를 기준으로 분리

In [241]:
# 데이터를 index가 같은 컬럼 별로 분리해서 리스트에 저장
# group: (그룹 이름, 그룹 데이터프레임) => group[1]: 그룹 데이터프레임
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
test_x_by_serial = [group[1] for group in test_x.groupby(test_x.index)]

# TIMESTAMP 값을 기준으로 정렬
train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
test_x_by_serial = [group.sort_values('TIMESTAMP') for group in test_x_by_serial]

In [242]:
train_x_by_serial[2]

,TIMESTAMP,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,
48,2020-02-09 08:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.845695,0.629975,0.059574,-1.734490,-0.465737,-0.012637,-0.035175,-0.464819,-0.010677,-0.011952,-0.013366
48,2020-02-10 08:16,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.849884,0.633251,0.059664,-1.684896,-0.410967,-0.012621,-0.035175,-0.409868,-0.010677,-0.011952,-0.012643
48,2020-02-11 08:31,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.854143,0.636589,0.059844,-1.635302,-0.315119,-0.012613,-0.035175,-0.313703,-0.010677,-0.011952,-0.012282
48,2020-02-12 08:46,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.858803,0.640521,0.060115,-1.583642,-0.219271,-0.012589,-0.035175,-0.217539,-0.010677,-0.011952,-0.011199
48,2020-02-13 09:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.863227,0.644140,0.060295,-1.534048,-0.127987,-0.012589,-0.035175,-0.125954,-0.010677,-0.011952,-0.011199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,2020-04-14 00:01,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.133848,0.867393,0.078570,1.480844,4.029984,-0.012373,-0.035175,4.045745,-0.010677,-0.011952,-0.001446
48,2020-04-15 00:16,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.138296,0.870981,0.078840,1.530438,4.198859,-0.012345,-0.035175,4.215177,-0.010677,-0.011952,-0.000362
48,2020-04-16 00:31,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.142649,0.874445,0.079110,1.580032,4.317527,-0.012345,-0.035175,4.334238,-0.010677,-0.011952,-0.000362


## train, valid 분할

In [243]:
def train_test_split(Xs, ys, test_ratio=0.2):
    ''' 각 (x, y) 쌍을 label 별로 딕셔너리에 저장 '''
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        # key: label, value: 해당 label에 해당하는 (x, y) 쌍의 리스트
        data_per_label[label].append((x, y))

    train = []
    test = []

    for label in data_per_label:
        # label에 해당하는 데이터 가져온다
        data = data_per_label[label]
        # 테스트 데이터의 개수
        n_test = int(len(data) * test_ratio)
        test += data[:n_test]
        train += data[n_test:]

    X_train, y_train = zip(*train)
    X_test, y_test = zip(*test)

    return X_train, X_test, y_train, y_test

In [244]:
X_train, X_val, y_train, y_val = train_test_split(train_x_by_serial, train_y['Y'], test_ratio=0.2)

# X data에서 Timestamp를 제거합니다.
X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in test_x_by_serial]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 6618
Validation Data의 개수 : 1654
Test Data의 개수 : 2069


## 머신러닝 모델에 적용하기 위해 학습, 검증, 테스트용 데이터를 각각 하나의 numpy array로 합칩니다.

In [245]:
def align_data(data, series_length):
    # X2~18열에 해당하는 시계열 데이터 값만 추출
    data_features = [x[feature_keys] for x in data]
    len_data = len(data_features)
    length_aligned_X = []
    for x in data_features:
        # 시계열 데이터 길이가 series_length 이상이면 > 뒷부분 잘라냄
        if len(x) >= series_length:
            length_aligned_X.append(x[:series_length])
        # 시계열 데이터 길이가 series_length보다 작으면 > 마지막 행을 반복하여 길이를 맞춤
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X)

In [246]:
series_length = 67

X_train = align_data(X_train, series_length)
X_val = align_data(X_val, series_length)
X_test = align_data(X_test, series_length)

In [247]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [248]:
X_train.shape

(6618, 67, 17)

# LGBM 학습

In [226]:
from lightgbm import LGBMClassifier

In [227]:
model = LGBMClassifier(num_leaves = 40,
                       min_child_samples = 60,
                       colsample_bytree = 0.8,
                       learning_rate = 0.05,
                       n_estimators = 500,
                       random_state = 42)
model.fit(X_train, y_train)

# 검증 데이터에 대한 예측
y_val_pred = model.predict(X_val)
# 검증 데이터에 대한 F1 점수 계산
f1_val = f1_score(y_val, y_val_pred, average='macro')
print('Validation F1 score = %.3f' % f1_val)

ValueError: Found array with dim 3. None expected <= 2.

# lstm 학습

In [228]:
!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [249]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

In [250]:
# X_train의 차원을 변경합니다.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# permute를 사용하여 차원을 (seq_length, batch_size, input_size)로 변환합니다.
X_train_tensor = X_train_tensor.permute(1, 0, 2)  # (64, 1139, 1) -> (1139, 64, 1)

# 데이터와 레이블을 Tensor로 변환
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)  # y_val은 클래스 레이블이므로 long 타입

X_val_tensor = X_val_tensor.permute(1, 0, 2)  # (batch_size, seq_length, 1) -> (seq_length, batch_size, 1)

In [251]:
from torch.utils.data import Dataset, DataLoader

class LabeledNumpyArrayDataset(Dataset):
    def __init__(self, numpy_data, numpy_labels, transform=None):
        self.data = numpy_data
        self.labels = numpy_labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        label = self.labels[index]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

In [252]:
batch_size = 32

train_dataset = LabeledNumpyArrayDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = LabeledNumpyArrayDataset(X_val_tensor, y_val_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [253]:
class LSTMModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, device):
        super(LSTMModel, self).__init__()

        self.device = device
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)


    def forward(self, data):
        batch_size = data.size(0)

        h0 = torch.zeros(self.layer_dim, batch_size, self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.layer_dim, batch_size, self.hidden_dim).to(self.device)

        out, (hn, cn) = self.lstm(data, (h0.detach(), c0.detach()))

        # 현재 out의 차원은 (batch_size, seq_length, hidden_size)입니다.
        # 이를 fully connected layer에 fit하게 차원을 변경(batch_size, hidden_size)해주어야 합니다.
        # fc layer를 통해 (batch_size, output_dim)로 차원을 변경해줍니다.
        out = self.fc(out[:, -1, :])

        return out

In [254]:
num_epoch = 10
hidden_dim = 4
layer_dim = 1
output_dim = 2
feature_size = X_train.shape[2]
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMModel(feature_size, hidden_dim, layer_dim, output_dim, 0.5, device)
model.apply(init_weights)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= 0.00002)
criterion = nn.CrossEntropyLoss()

In [255]:
for epoch in range(num_epoch):
    model.train()
    for i, (text, labels) in enumerate(train_dataloader):
    
        text = text.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        logits = model(text)
        loss = criterion(logits, labels)
        loss.backward()

        # Updating parameters
        optimizer.step()

In [256]:
# 1. 모델을 평가 모드로 설정
model.eval()

# 2. 모든 배치에서 예측을 저장할 리스트
all_preds = []
all_labels = []

# 3. 검증 데이터에 대해 예측 수행
with torch.no_grad():
    for inputs, labels in val_dataloader:  # valid_dataloader는 검증 데이터 로더
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델 예측
        logits = model(inputs)
        _, preds = torch.max(logits, 1)  # 가장 높은 확률을 가진 클래스 선택

        # 결과를 리스트에 저장
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 4. F1 점수 계산
f1 = f1_score(all_labels, all_preds, average='macro')  # 'weighted'는 클래스 불균형을 고려한 평균 방식

print(f'F1 Score: {f1:.4f}')


F1 Score: 0.4640


In [257]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# permute를 사용하여 차원을 (seq_length, batch_size, input_size)로 변환합니다.
#X_test_tensor = X_test_tensor.permute(1, 0, 2)  


# 2. 모델 평가 모드로 설정
model.eval()

# 3. 예측 수행
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    logits = model(X_test_tensor)
    _, y_test_pred = torch.max(logits, 1)  # 가장 높은 확률을 가진 클래스 선택

# 4. 예측 결과 후처리
submission = pd.read_csv(os.path.join(data_dir, "test_y.csv"), index_col='Serial Number')
submission["Y"] = y_test_pred.cpu().numpy()  # 예측 결과를 넘파이 배열로 변환하여 저장
submission.to_csv("submission.csv", index_label='Serial Number')
submission
# 예측 결과 중 1의 비율을 계산하고, 학습용 데이터의 비율과 비교합니다.
print(submission["Y"].mean())
print(train_y.mean())

0.09569840502658289
Y    0.146518
dtype: float64


### 제출

우측 상단의 제출 버튼을 눌러, `competition.ipynb` 파일과 `submission.csv` 파일을 제출합니다.